In [1]:
from ModifiedUnet import ModifiedUnet

/opt/anaconda3/envs/ECE4179/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/opt/anaconda3/envs/ECE4179/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
model = ModifiedUnet()


for name , param in model.named_parameters():
    print(name , param.shape)

An error occurred while trying to fetch segmind/tiny-sd: segmind/tiny-sd does not appear to have a file named diffusion_pytorch_model.safetensors.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


unet.conv_in.weight torch.Size([320, 4, 3, 3])
unet.conv_in.bias torch.Size([320])
unet.time_embedding.linear_1.weight torch.Size([1280, 320])
unet.time_embedding.linear_1.bias torch.Size([1280])
unet.time_embedding.linear_2.weight torch.Size([1280, 1280])
unet.time_embedding.linear_2.bias torch.Size([1280])
unet.down_blocks.0.attentions.0.norm.weight torch.Size([320])
unet.down_blocks.0.attentions.0.norm.bias torch.Size([320])
unet.down_blocks.0.attentions.0.proj_in.weight torch.Size([320, 320, 1, 1])
unet.down_blocks.0.attentions.0.proj_in.bias torch.Size([320])
unet.down_blocks.0.attentions.0.transformer_blocks.0.norm1.weight torch.Size([320])
unet.down_blocks.0.attentions.0.transformer_blocks.0.norm1.bias torch.Size([320])
unet.down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_q.base_layer.weight torch.Size([320, 320])
unet.down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_q.lora_A.default.weight torch.Size([64, 320])
unet.down_blocks.0.attentions.0.transformer_blocks

In [3]:
from PIL import Image
import torch
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, PNDMScheduler


vae = AutoencoderKL.from_pretrained("segmind/tiny-sd", subfolder="vae")
tokenizer = CLIPTokenizer.from_pretrained("segmind/tiny-sd", subfolder="tokenizer")
text_encoder = CLIPTextModel.from_pretrained("segmind/tiny-sd", subfolder="text_encoder")

vae/config.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

An error occurred while trying to fetch segmind/tiny-sd: segmind/tiny-sd does not appear to have a file named diffusion_pytorch_model.safetensors.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


diffusion_pytorch_model.bin:   0%|          | 0.00/167M [00:00<?, ?B/s]

/opt/anaconda3/envs/ECE4179/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer/tokenizer_config.json:   0%|          | 0.00/737 [00:00<?, ?B/s]

...

tokenizer/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer/special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

text_encoder/config.json:   0%|          | 0.00/722 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/246M [00:00<?, ?B/s]

/opt/anaconda3/envs/ECE4179/lib/python3.11/site-packages/transformers/modeling_utils.py:519: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file,

In [4]:
from diffusers import UniPCMultistepScheduler

scheduler = UniPCMultistepScheduler.from_pretrained("segmind/tiny-sd", subfolder="scheduler")

scheduler/scheduler_config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

## HYPERPARAMETERS

In [5]:
MAX_EPOCH = 50
HEIGHT = 512  # default height of Stable Diffusion ( IMAGE HEIGHT)
WIDTH = 512  # default width of Stable Diffusion    ( IMAGE WIDTH)

NUM_INFERENCE_STEPS = 25 # Number of denoising steps
GUIDANCE_SCALE = 7.5  # Scale for classifier-free guidance
GENERATOR = torch.manual_seed(0)  # Seed generator to create the inital latent noise
 
 
 
 # CHANGE IF NEEDED TO 
BATCH_SIZE = 8 
LEARNING_RATE = 1e-5 # 



optimizer = ??
loss_fn = ??

## TRAINING



In [6]:
import tqdm

In [ ]:
for i in range(MAX_EPOCH):
    for batch in tqdm(dataloader):
        images , prompts = batch 

        text_input = tokenizer(
            prompts, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt"
        )
        
        max_length = text_input.input_ids.shape[-1]
        uncond_input = tokenizer([""] * BATCH_SIZE, padding="max_length", max_length=max_length, return_tensors="pt")
        with torch.no_grad():
            text_embeddings = text_encoder(text_input.input_ids)[0]
            uncond_embeddings = text_encoder(uncond_input.input_ids)[0]

        text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

        latents = torch.randn(
            (BATCH_SIZE, model.in_channels, HEIGHT // 8, WIDTH // 8),
            generator=GENERATOR,
        )
        latents = latents * scheduler.init_noise_sigma

        original_latents = vae(images)  # Replace `encoder` with the actual function you use


        for t in tqdm(scheduler.timesteps):

            # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
            latent_model_input = torch.cat([latents] * 2)

            latent_model_input = scheduler.scale_model_input(latent_model_input, timestep=t)

            # predict the noise residual
            noise_pred = model(latent_model_input, t, encoder_hidden_states=text_embeddings).sample

            actual_noise = (latents - original_latents) / scheduler.init_noise_sigma

            # perform guidance
            noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
            noise_pred = noise_pred_uncond + GUIDANCE_SCALE * (noise_pred_text - noise_pred_uncond)

            # compute the previous noisy sample x_t -> x_t-1
            latents = scheduler.step(noise_pred, t, latents).prev_sample
            
            
            loss = loss_fn(noise_pred, actual_noise)  # Define your loss function
            optimizer.zero_grad()  # Clear previous gradients
            loss.backward()        # Compute gradients
            optimizer.step()
